# Création du Dataset en scrappant les infos depuis une liste d'url

## Imports

In [ ]:
import scraping_info_perfume as sip
import pandas as pd
import numpy as np
import time
import sys

df = pd.read_csv('list_url_by_popular_designer.csv')

# Choix de la lettre à scraper
lettre = 'T'
df_a_scraper = df[df['Marque'].str.startswith(lettre)] 
df_a_scraper.describe()

,Marque,Parfum,Lien du Parfum
count,19318,19318,19318
unique,119,18643,19318
top,The Dua Brand,Rose,https://www.fragrantica.com/perfume/Acqua-di-P...
freq,1673,8,1


## Test pour voir le temps d'attente optimal à scraper 50 parfums

In [ ]:
batch_size = 49

# Charger le dataset existant
Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
url_deja_scrape = set(Fragrantica_dataset['url'].tolist())

# Filtrer les URLs déjà scrapées
df_a_scraper = df_a_scraper[~df_a_scraper['Lien du Parfum'].isin(url_deja_scrape)]

# Initialiser les DataFrames pour les données scrapées
scraped_data_1 = pd.DataFrame()
scraped_data_2 = pd.DataFrame()

# Fonction pour scraper un batch et mesurer le temps d'exécution
def scrape_batch(batch, batch_num):
    liste_url = batch["Lien du Parfum"].tolist()
    start_time = time.time()
    scraped_data = sip.scraping_multi_perfume_info(liste_url)
    end_time = time.time()
    execution_time = end_time - start_time
    scraped_data.to_csv(f"scraped_data_{batch_num}.csv", index=False)
    return scraped_data, execution_time

# Scraper le premier batch
batch1 = df_a_scraper.iloc[0 : batch_size]
scraped_data_1, execution_time_1 = scrape_batch(batch1, 1)
print(f"Temps d'exécution pour batch 1 : {execution_time_1:.1f} secondes.")
print(f"Soit {execution_time_1 / batch_size:.1f} secondes par URL.")

# Attendre avant de scraper le deuxième batch
time.sleep(180)

# Scraper le deuxième batch
batch2 = df_a_scraper.iloc[batch_size : 2 * batch_size]
scraped_data_2, execution_time_2 = scrape_batch(batch2, 2)
print(f"Temps d'exécution pour batch 2 : {execution_time_2:.1f} secondes.")
print(f"Soit {execution_time_2 / batch_size:.1f} secondes par URL.")


Temps d'exécution pour batch 1 : 44.2 secondes.
Soit 8.8 secondes par URL.
Temps d'exécution pour batch 2 : 72.0 secondes.
Soit 14.4 secondes par URL.


In [12]:

Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
print(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")
scraped_data1 = pd.read_csv(f"scraped_data_1.csv")
#afficher le nomre de ligne de chaque dataframe
# Concaténer les deux DataFrames
combined_data = pd.concat([Fragrantica_dataset, scraped_data1])
combined_data.drop_duplicates(subset=['url'], inplace=True)
combined_data.to_csv("Fragrantica_dataset.csv", index=False)

Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
print(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")
scraped_data2 = pd.read_csv(f"scraped_data_2.csv")
combined_data = pd.concat([Fragrantica_dataset, scraped_data2])
combined_data.drop_duplicates(subset=['url'], inplace=True)
combined_data.to_csv("Fragrantica_dataset.csv", index=False)


Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
print(f"Nombre de lignes dans Fragrantica_dataset : {Fragrantica_dataset.shape[0]}")
print(f"✅ Les données ont été ajoutées à Fragrantica_dataset.csv sans doublons.")




Nombre de lignes dans Fragrantica_dataset : 578
Nombre de lignes dans Fragrantica_dataset : 578
Nombre de lignes dans Fragrantica_dataset : 588
✅ Les données ont été ajoutées à Fragrantica_dataset.csv sans doublons.


## Le script qui scrape les infos par batch de 49 lignes

In [ ]:
batch_size = 49
num_batches = len(df_a_scraper) // batch_size + (1 if len(df_a_scraper) % batch_size != 0 else 0)
print(f"📦 TAILLE DES BATCHS = {batch_size}")

time.sleep(3600)

#1) Obtenir les URLs déjà scrapées
Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")
url_deja_scrape = set(Fragrantica_dataset['url'].tolist())  # Convertir en set pour accélérer la recherche
print(f"📂 Nous avons déja {len(url_deja_scrape)} URLs sont dans le dataset Fragrantica.")
print("--------------------------------------------------------------------------------")

num_url_scrap = 0
#2) Itérer sur les batches 
for i in range(num_batches):
    print(f"\n📡 Traitement du batch {i+1}/{num_batches}...")

    batch = df_a_scraper.iloc[i * batch_size : (i + 1) * batch_size]
    liste_url = batch["Lien du Parfum"].tolist()

    # 🔍 Supprimer les URLs déjà scrapées
    old_len = len(liste_url)
    liste_url = [url for url in liste_url if url not in url_deja_scrape]
    new_len = len(liste_url)
    print(f"🔍 {old_len - new_len} URLs sur {old_len} ont déjà été scrapées et retirées du batch {i+1}.")

    if not liste_url:  # Si aucun URL à scraper, passer au batch suivant
        print(f"⏭️ Batch {i+1} ignoré (toutes les URLs étaient déjà scrapées).")
        continue
    

    try:
        start_time = time.time()
        scraped_data = sip.scraping_multi_perfume_info(liste_url)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"⏱️ Temps d'exécution du scraping sur le batch{i+1} : {execution_time:.1f} secondes.")
        print(f"⏱️ Soit un total de {execution_time/len(liste_url):.1f} secondes par URL.")
        
        # Vérifier si le batch est vide

        '''
        cette condition n'est jamais remplie
        '''
        if scraped_data['nom_parfum'].empty:
            print(f"🚨 Erreur critique : Le batch {i+1} est vide. On fait une grosse pause")
            time.sleep(3600)  # Pause de 5 minutes pour éviter d'être bloqué par le site
            continue
        # Vérifier si le scrappinng c'es mal passé
        if scraped_data['nom_parfum'].isnull().sum() > 0:
            print(f"🚨 Erreur critique : Le batch {i+1} contient {scraped_data['nom_parfum'].isnull().sum()} lignes vides. On fait une grosse pause")
            # Filtrer les lignes valides
            scraped_data = scraped_data.dropna(subset=['nom_parfum'])
    
            file_name = f"scrap_batch_{i+1}.csv"
            scraped_data.to_csv(file_name, index=False) # Sauvegarde du batch sans index inutile
            print(f"✅ Batch {i+1} enregistré dans {file_name} avec les lignes valides.")
            num_url_scrap += len(scraped_data)
            time.sleep(3600)  # Pause de 5 minutes pour éviter d'être bloqué par le site
            continue
            '''        
            print(f"🚨 Erreur critique : Le batch {i+1} contient des lignes vides. Arrêt immédiat du script.")
            sys.exit(1)  # 💀 Arrêt immédiat du script avec un code d'erreur 1
            '''

        else:
            file_name = f"scrap_batch_{i+1}.csv"
            scraped_data.to_csv(file_name, index=False) # Sauvegarde du batch sans index inutile
            print(f"✅ Batch {i+1} enregistré dans {file_name}.")
            num_url_scrap += new_len

        print("On attend 30 minutes")
        time.sleep(1801)  # Pause pour éviter d'être bloqué par le site

    except Exception as e:
        print(f"⚠️ Erreur lors du scraping du batch {i+1}: {e}")

'''
Fais une grosse pause seulement si on a une 'Too many requests' erreur

'''



print(f"\n🎉 AU FINAL {num_url_scrap} URLs ont été scrapées avec succès !")

#3) Regarder le dernier batch
scraped_data = pd.DataFrame()
scraped_data.head()



📦 TAILLE DES BATCHS = 49


## Code pour ajouter nos nouvelles lignes à 'Fragrantica_dataset.csv' 

In [ ]:
num_premier_batch = 1
num_dernier_batch = 2
for i in range(num_premier_batch-1, num_dernier_batch):
    scraped_data = pd.read_csv(f"scrap_batch_{i+1}.csv")
    Fragrantica_dataset = pd.read_csv("Fragrantica_dataset.csv")

    # Concaténer les deux DataFrames
    combined_data = pd.concat([Fragrantica_dataset, scraped_data])

    # Supprimer les doublons
    combined_data.drop_duplicates(subset=['url'], inplace=True)

    # Sauvegarder le dataset combiné
    combined_data.to_csv("Fragrantica_dataset.csv", index=False)
    print(f"✅ Les données du batch {i+1} ont été ajoutées à Fragrantica_dataset.csv sans doublons.")


✅ Les données du batch 18 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 19 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 20 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 21 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 22 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 23 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 24 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 25 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 26 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 27 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 28 ont été ajoutées à Fragrantica_dataset.csv sans doublons.
✅ Les données du batch 29 ont été ajoutées à Fragrantica_dataset.csv sans do